# Chapter 13: Operator Overloading: Doing It Right

There are different kinds of operators in Python:

* An operator that requires two operands is a binary operator. All the binary operators use **infix** notation, which means that the operator appears between its operands. `+` and `|` are examples of **infix operators**.

* An operator that requires one operand is called a **unary operator**. `-` and `~` are examples of unary operators.

* `()` (function invocation), `.` (attribute access) and `[]` (item access/slicing) are also examples of operators.

This chapter deals mainly with infix and unary operators, and will cover:

* How Python supports infix operators with operands of different types.

* Using duck typing or explicit type checks to deal with operands of various types.

* How an infix operator method should signal it cannot handle an operand.

* The special behaviour of the rich comparison operators (e.g., `==`, `>`, `<`, `<=`, etc.).

* The default handling of augmented assignment operators, like `+=`, and how to overload them.

## Operator Overloading 101

There are some limitations imposed on operator overloading in Python. **We cannot:**

* overload the operators of built-in types.
* create new operators (only overload existing).
* overload `is`, `and`, `or` and `not`.

The rich comparison operators (e.g. `==`, `!=`, `>`, `<`, etc.) are special cases in operator overloading, which will be covered later.

## Unary Operators

*The Python Language Reference* lists three unary operators and their associated special methods:

`-` (`__neg__`): Arithmetic unary negation. If `x == -2` then `-x == 2`.

`+` (`__pos__`): Arithmetic unary plus. `x == +x`, excluding some special cases (see **p. 388**).

`~` (`__invert__`): Bitwise inverse of an integer. Defined as `~x == -(x+1)`If `x == 2` then `~x == -3`.

*The Python Language Reference* also lists `abs(...)` as a unary operator. The associated special method is `__abs__`.

The unary operators are easily implemented by defining the above special methods (requires only `self` argument).

However, remember the **fundamental rule of operators**: Always return a new object, i.e. create and return a new instance of a suitable type. **Special methods implementing unary or infix operators should never change their operands.**

The exception to this rule is `+`, where it is the best approach is (usually) to return a copy of `self`.

For `abs(...)` the result should be a scalar number.

In [1]:
# Example 13-1. vector_v6.py: unary operators - and +

def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

def __neg__(self):
    return Vector(-x for x in self)

def __pos__(self):
    return Vector(self)

## Overloading `+` for Vector Addition

According to the [Data Model chapter](https://docs.python.org/3/reference/datamodel.html) in the official docs, sequences should support the `+` operator for concatenation and `*` operator for repetition.

In our `Vector` class we will instead implement these as mathematical vector operations, which makes more sense for the type.

We will also allow for addition of vectors of different lengths, due to practical considerations (may be used for information retrieval, etc.). To facilitate this, we use `zip_longest` and fill out the shortest `Vector` with zeros.

In [2]:
# Example 13-4. Vector.add method, take #1

import itertools

def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fill_value=0.0)
    return Vector(a + b for a, b in pairs)

Note that this implementation allows for addition of a `Vector` with any iterable, because `zip_longest` can consume any iterable.

In [3]:
from examples.vector_v6 import Vector

v = Vector(range(4))
v + [1, 2, 3, 4, 5, 6]

Vector([1.0, 3.0, 5.0, 7.0, 5.0, ...])

However, if the operands are reversed, this fails:

In [4]:
[1, 2, 3, 4, 5, 6] + v

TypeError: can only concatenate list (not "Vector") to list

This happens because Python implements a special dispatching mechanism for the infix operator special methods. 

Given `a + b`, the interpreter performs the following steps:

1. If `a` has `__add__`, call `a.__add__(b)` and return unless it is `NotImplemented`.
2. If `a` does not have `__add__`, or calling it returns `NotImplemented`, check if `b` has `__radd__`, then call `b.__radd__(a)` and return result unless it is not implemented.
3. If `b` does not have `__radd__`, or calling it returns `NotImplemented`, raise `TypeError` with an *unsupported operand types* message.

The `__radd__` method is called the **"reflected"** or **"reversed"** version of `__add__`.

To make the mixed-type addition work in `Vector`, we must implement the `__radd__` special method. Note that this is as simple as delegating to the `__add__` method.

In [5]:
# Example 13-4. Vector.__add__ and .__radd__ methods

import itertools

def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fill_value=0.0)
    return Vector(a + b for a, b in pairs)

def __radd__(self, other):
    return self + other

The above methods work with any iterable with numeric items, but fails when provided with a non-iterable object or iterables with non-numeric items:

In [6]:
v + 1

TypeError: zip_longest argument #2 must support iteration

In [7]:
v + "ABC"

TypeError: unsupported operand type(s) for +: 'float' and 'str'

If an operator cannot return a valid result because of type incompatibility, it should return `NotImplemented` instead of raising a `TypeError`. 

By returning `NotImplemented`, you leave the door open for the implementer of the other operand type to perform the operation when Python tries the reversed method call.

**In the spirit of duck typing, we do not test the input type of `other`, and instead catch exceptions and return `NotImplemented`.**

If the reverse method call returns `NotImplemented`, then Python will raise `TypeError` with a standard error message.

In [8]:
# Example 13-10. vector_v6_final.py: final operator + methods

def __add__(self, other):
    try:
        pairs = itertools.zip_longest(self, other, fill_value=0.0)
        return Vector(a + b for a, b in pairs)
    except TypeError:
        raise NotImplemented
        
def __radd__(self, other):
    return self + other

**Takeaway:** If an infix operator raises an exception, it aborts the operator dispatch algorithm. In the particular case of `TypeError`, it is often better to catch it and return `NotImplemented`. This allows the interpreter to  try calling the reversed operator method, which may correctly handle the computation with the swapped operands, if the are of different types.

## Overloading `*` for Scalar Multiplication

There should be two kinds of multiplications available for `Vector` instances: scalar multiplication and dot products.

In the case of scalar multiplications, Ramalho shows a practical example of goose typing (an explicit check against an abstract type, see [chapter 11]()):

In [9]:
# inside the Vector class (see vector_v7.py)

import numbers

def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):
        return Vector(scalar * x for x in self)
    else:
        raise NotImplemented
        
def __rmul__(self, scalar):
    return self * scalar

def __matmul__(self, other):
    try:
        return sum(a * b for a, b in zip(self, other))
    else:
        raise NotImplemented
        
def __rmatmul__(self, other):
    return self @ other

SyntaxError: invalid syntax (<ipython-input-9-e90464a95449>, line 17)

Above we use `isinstance()` to check the type of `scalar`, but instead of hardcoding some concrete types, we check against the `numbers.Real` ABC, which covers all the types we need.

This keeps our implementation open to future numeric types that declare themselves as actual or virtual subclasses of the `numbers.Real` ABC.

The patterns used when implementing `+` and `*` illustrate the most common patterns for coding infix operators, and are applicable to all operators listed in table 13-1 on **p. 397**.

## Rich Comparison Operators

The handling of rich comparison operators by the Python interpreter is similar to those described above, but differs in two important ways:

* The same set of methods are used in forward and reverse operator calls, summarised in the table below.
* In the case of `==` and `!=`, if the reverse call fails, Python compares the object IDs instead of raising `TypeError`.

| Group    | Infix operator | Forward method call | Reverse method call | Fall back               |
| -------- | -------------- | ------------------- | ------------------- | ----------------------- |
| Equality | `a == b`       | `a.__eq__(b)`       | `b.__eq__(a)`       | Return `id(a) == id(b)` |
|          | `a != b`       | `a.__ne__(b)`       | `b.__ne__(a)`       | Return `not (a == b)`   |
| Ordering | `a > b`        | `a.__gt__(b)`       | `b.__lt__(a)`       | Raise `TypeError`       |
|          | `a < b`        | `a.__lt__(b)`       | `b.__gt__(a)`       | Raise `TypeError`       |
|          | `a >= b`       | `a.__ge__(b)`       | `b.__le__(a)`       | Raise `TypeError`       |
|          | `a <= b`       | `a.__le__(b)`       | `b.__ge__(a)`       | Raise `TypeError`       |

Recall the original implementation of `Vector.__eq__`:

```python
def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))
```

This method produces the following results:

In [10]:
from examples.vector2d_v3 import Vector2d
from examples.vector_v5 import Vector

va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
vc = Vector([1, 2])
v2d = Vector2d(1, 2)
t3 = (1, 2, 3)

In [11]:
print("va == vb:", va == vb)
print("vc == v2d:", vc == v2d)
print("va == t3:", va == t3)

va == vb: True
vc == v2d: True
va == t3: True


The last result is probably not desirable.

To solve this, Ramalho suggests explicitly type checking to be conservative. Using the rules listed in the table above, we improve `Vector.__eq__`:

In [12]:
# in Vector class

def __eq__(self, other):
    if isinstance(other, Vector):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    else:
        raise NotImplemented

In [13]:
from examples.vector2d_v3 import Vector2d
from examples.vector_v8 import Vector

va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
v2d = Vector2d(1, 2)
t3 = (1, 2, 3)

print("va == vb:", va == vb)
print("vc == v2d:", vc == v2d)
print("va == t3:", va == t3)

va == vb: True
vc == v2d: True
va == t3: False


In this case, the reversed call delegates the equality check to a `tuple`, which compares object IDs as a fall back, returning `False`.

Because of the fall back behaviour of `__ne__` (see table above), we don't need to implement it in `Vector` (`not (a == b) -> not (False)`).

## Augmented Assignment Operators

If you have implemented `__add__` then `+=` will work with no additional code.

This is because if a class does not implement the in-place operators listed in table 13-1, the augmented assignment operators are just syntactic sugar: `a += b` is evaluated as `a = a + b`.

However, if an in-place operator such as `__iadd__` is implemented, that method is called to compute the result of `a += b`. **It is very important that augmented assignment special methods return `self`.**

As the name suggests, these operators are expected to change the left hand operator *in place* and not create a new object.

**In-place methods should never be implemented in immutable types, like our `Vector` class.**

To show the implementation of an in-place operator, we extend `BingoCage` from example 11-12:

In [14]:
# Example 13-18. bingoaddable.py: AddableBingoCage extends BingoCage to support + and +=

import itertools
from examples.tombola import Tombola
from examples.bingo import BingoCage


class AddableBingoCage(BingoCage):
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented  # noqa: F901

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = f"right operand in += must be {self_cls!r} or an iterable"
                raise TypeError(msg)
        self.load(other_iterable)
        return self


In [15]:
first_globe = AddableBingoCage([1, 2, 3])
globe_orig = first_globe
second_globe = AddableBingoCage([4, 5])
print("Original length:", len(first_globe.inspect()))

first_globe += second_globe 
print("Length after in-place addition:", len(first_globe.inspect()))
print("Check if object returned is the same:", first_globe is globe_orig)

print("\nWe expect this to fail:")
first_globe += 1

Original length: 3
Length after in-place addition: 5
Check if object returned is the same: True

We expect this to fail:


TypeError: right operand in += must be 'AddableBingoCage' or an iterable

## Comments to chapter

A final note in this chapter is that if a forward method only deals with righthand operands of the same type, then there is **no need to implement a reverse method.**

This chapter is slightly inconsistent. In the chapter summary, Ramalho writes:

> Mixing operand types means we need to detect when we get an operand we can't handle. In this chapter, we did this in two ways: in the duck typing way, we just went ahead and tried the operation, catching a `TypeError` exception if it happened; later, we [...] did it with an explicit `isinstance` test. [...] When we did use `isinstance`, we were careful to avoid testing with a concrete class [...].

However, in the `Vector` example above, we explicitly test against a concrete class.